In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import io
import re
import seaborn as sns

In [2]:
# === CONFIGURAÇÕES ===
# Caminho para seu CSV (pode ser o summary consolidado)
CSV_PATH = "./summary_results.csv"

In [3]:
# === LEITURA E LIMPEZA ===
# Tenta ler diretamente tratando vírgula decimal; se falhar, faz fallback lendo o arquivo e substituindo vírgula por ponto.

try:
    df = pd.read_csv(CSV_PATH, encoding='utf-8', decimal=",")
except Exception:
    with open(CSV_PATH, "r", encoding="utf-8") as f:
        content = f.read().replace(",", ".")
    df = pd.read_csv(io.StringIO(content))
# === PRÉ-PROCESSAMENTO ===
# Garante nomes consistentes
df.columns = [c.strip().replace('"', '') for c in df.columns]
# Remove espaços, corrige tipos numéricos
for col in ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Extrai linguagem e teste do nome do executável (ex: tc1_HS.exe)
def parse_test_lang(filename):
    base = str(filename).replace(".exe", "").replace(".EXE", "")
    parts = base.split("_")
    if len(parts) >= 2:
        test = "_".join(parts[:-1])
        lang = parts[-1]
        return test, lang
    return base, "Unknown"

df[["Test", "Language"]] = df["File"].apply(lambda f: pd.Series(parse_test_lang(f)))

# Extrai nível de otimização (o1, o2, o3...) do nome do assembly
df["Optimization"] = df["AsmFile"].apply(
    lambda x: re.search(r"o(\d+)", str(x)).group(1) if re.search(r"o(\d+)", str(x)) else "0"
)


In [4]:
# === DEBUG ===
print("\n=== DADOS FORMATADOS ===")
print(df.head(5))
print("\nLinguagens:", df["Language"].unique())
print("Testes:", df["Test"].unique())
print("Otimizacoes:", df["Optimization"].unique())

# === VISÃO GERAL ===
print("\n=== RESUMO DOS DADOS ===")
print(df.head())
print("\nLinguagens:", df['Language'].unique())
print("Testes:", df['Test'].unique())
print("Assembly Files:", df['AsmFile'].unique())



=== DADOS FORMATADOS ===
             File  ExecTime_ms  MaxWorking_MB  MaxPrivate_MB  AvgCPU_pct  \
0  nr_iter_py.exe      1364.61          6.410          1.352        3.78   
1  nr_iter_py.exe      1356.28          6.406          1.363        3.91   
2  nr_iter_py.exe      1408.82          6.406          1.348        4.02   
3  nr_iter_py.exe      1365.94          6.641          1.367        3.84   
4  nr_iter_py.exe      1317.90          6.402          1.355        3.84   

           AsmFile  AsmLines     Test Language Optimization  
0  o3_nr_iter_py.s         0  nr_iter       py            3  
1  o3_nr_iter_py.s         0  nr_iter       py            3  
2  o3_nr_iter_py.s         0  nr_iter       py            3  
3  o3_nr_iter_py.s         0  nr_iter       py            3  
4  o3_nr_iter_py.s         0  nr_iter       py            3  

Linguagens: ['py' 'hs' 'c']
Testes: ['nr_iter' 'nr_recursive']
Otimizacoes: ['3']

=== RESUMO DOS DADOS ===
             File  ExecTime_ms  MaxW

In [5]:
# === FUNÇÃO DE COMPARAÇÃO (adiciona boxplots, violin plots e gráficos de linha) ===
def compare_test2(test_name):
    """Gera gráficos comparando C e Haskell para um teste específico (usa média por Optimization).
       Além dos gráficos agregados, gera boxplots, violin plots e gráficos de linha com os dados individuais por nível de otimização."""
    subset = df[df["Test"] == test_name].copy()

    if subset.empty:
        print(f"⚠️ Nenhum dado encontrado para o teste '{test_name}'")
        return

    metrics = ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]
    metric_labels = {
        "ExecTime_ms": "Tempo de Execução (ms)",
        "MaxWorking_MB": "Memória Física Máx (MB)",
        "MaxPrivate_MB": "Memória Privada Máx (MB)",
        "AvgCPU_pct": "Uso de CPU (%)",
        "AsmLines": "Linhas Assembly"
    }

    # Garantir que Optimization seja ordenável (int) para um eixo X correto
    subset["OptLevel"] = pd.to_numeric(subset["Optimization"], errors="coerce").fillna(-1).astype(int)
    subset.sort_values("OptLevel", inplace=True)
    subset["Optimization_str"] = subset["OptLevel"].astype(str)

    # Linguagens e níveis de otimização existentes — manter ordem consistente
    langs = sorted(subset["Language"].unique())
    opt_levels = sorted(subset["OptLevel"].unique())
    opt_labels = [str(x) for x in opt_levels]

    # Para cada métrica, plotar a média por (Optimization, Language) com barras (e erro padrão) e linhas
    for metric in metrics:
        agg = subset.groupby(["OptLevel", "Language"])[metric].agg(["mean", "std", "count"]).reset_index()

        # Reindex para incluir combinações ausentes (faz aparecer a legenda/trace mesmo se houver NaNs)
        if not agg.empty:
            idx = pd.MultiIndex.from_product([opt_levels, langs], names=["OptLevel", "Language"])
            agg = agg.set_index(["OptLevel", "Language"]).reindex(idx).reset_index()
            agg["Optimization"] = agg["OptLevel"].astype(str)
            # manter count como 0 onde não há dados
            agg["count"] = agg["count"].fillna(0)

            # Bar chart (média por optimization)
            fig = px.bar(
                agg,
                x="Optimization",
                y="mean",
                color="Language",
                barmode="group",
                error_y="std",
                labels={"mean": metric_labels[metric]},
                title=f"{test_name} — {metric_labels[metric]} (média por nível de otimização) — Barras",
                category_orders={"Optimization": opt_labels, "Language": langs}
            )
            fig.update_layout(
                template="plotly_white",
                font=dict(size=14),
                yaxis_title=metric_labels[metric],
                xaxis_title="Nível de Otimização"
            )
            fig.show()

            # Line chart (média por optimization)
            fig_line = px.line(
                agg,
                x="Optimization",
                y="mean",
                color="Language",
                markers=True,
                labels={"mean": metric_labels[metric]},
                title=f"{test_name} — {metric_labels[metric]} (média por nível de otimização) — Linha",
                category_orders={"Optimization": opt_labels, "Language": langs}
            )
            fig_line.update_traces(mode="lines+markers", connectgaps=True)
            fig_line.update_layout(
                template="plotly_white",
                font=dict(size=14),
                yaxis_title=metric_labels[metric],
                xaxis_title="Nível de Otimização"
            )
            fig_line.show()

    # Boxplots e Violin plots usando dados individuais (por nível de otimização e linguagem)
    for metric in metrics:
        if metric not in subset.columns:
            continue
        # Boxplot
        fig_box = px.box(
            subset,
            x="Optimization_str",
            y=metric,
            color="Language",
            points="outliers",
            labels={metric: metric_labels[metric], "Optimization_str": "Nível de Otimização"},
            title=f"{test_name} — Boxplot de {metric_labels[metric]} por Nível de Otimização",
            category_orders={"Optimization_str": opt_labels, "Language": langs}
        )
        fig_box.update_layout(template="plotly_white", font=dict(size=13))
        fig_box.show()

        # Violin plot (com box interno e pontos)
        fig_violin = px.violin(
            subset,
            x="Optimization_str",
            y=metric,
            color="Language",
            box=True,
            points="all",
            labels={metric: metric_labels[metric], "Optimization_str": "Nível de Otimização"},
            title=f"{test_name} — Violin de {metric_labels[metric]} por Nível de Otimização",
            category_orders={"Optimization_str": opt_labels, "Language": langs}
        )
        fig_violin.update_layout(template="plotly_white", font=dict(size=13))
        fig_violin.show()

    # Comparativo médio por linguagem (usado para resumo, radar e speed ratio)
    mean_data = subset.groupby("Language")[metrics].mean().reset_index()
    print(f"=== MÉDIAS — {test_name} ===")
    print(mean_data)
    print()

    # Radar Chart (perfil de desempenho) - normalização segura (evita divisão por zero)
    normalized = mean_data.copy()
    for col in metrics:
        mn = mean_data[col].min()
        mx = mean_data[col].max()
        if pd.isna(mn) or pd.isna(mx) or mx == mn:
            normalized[col] = 0.5
        else:
            normalized[col] = (mean_data[col] - mn) / (mx - mn)

    fig_radar = go.Figure()
    for _, row in normalized.iterrows():
        fig_radar.add_trace(go.Scatterpolar(
            r=row[metrics].values,
            theta=[metric_labels[m] for m in metrics],
            fill="toself",
            name=row["Language"]
        ))
    fig_radar.update_layout(
        title=f"Perfil de Desempenho Normalizado — {test_name}",
        polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
        template="plotly_white"
    )
    fig_radar.show()

    # Speed Ratio (quanto Haskell é mais lento/rápido que C) usando médias
    if {"c", "hs"}.issubset(set(mean_data["Language"])):
        c_data = mean_data.loc[mean_data["Language"] == "c"].iloc[0]
        hs_data = mean_data.loc[mean_data["Language"] == "hs"].iloc[0]
        py_data = mean_data.loc[mean_data["Language"] == "py"].iloc[0] if "py" in mean_data["Language"].values else None

        if c_data["ExecTime_ms"] == 0:
            print(f"⚠️ ExecTime_ms média de C é zero para {test_name}, não é possível calcular ratio.")
        else:
            ratio = hs_data["ExecTime_ms"] / c_data["ExecTime_ms"]
            print(f"⚡ Speed Ratio ({test_name}): Haskell é {ratio:.2f}x o tempo de C.\n")

# === EXECUÇÃO DAS COMPARAÇÕES PARA CADA TESTE ===
for test in df["Test"].unique():
    compare_test2(test)

C:\Users\maria\AppData\Roaming\Python\Python313\site-packages\kaleido\__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




=== MÉDIAS — nr_iter ===
  Language  ExecTime_ms  MaxWorking_MB  MaxPrivate_MB  AvgCPU_pct  AsmLines
0        c   141.849099       2.063955       0.441261    0.305135     135.0
1       py  1448.436818       6.423464       1.369009    3.644364       0.0



=== MÉDIAS — nr_recursive ===
  Language  ExecTime_ms  MaxWorking_MB  MaxPrivate_MB  AvgCPU_pct  AsmLines
0        c   150.186514       2.011752       0.432578    0.254954     172.0
1       hs   157.874019       2.299131       3.306579    0.946355     576.0
2       py   753.265727       6.355264       1.338155    2.754182       0.0



⚡ Speed Ratio (nr_recursive): Haskell é 1.05x o tempo de C.



In [6]:
def compare_tests_aggregated(df):
    """
    Gera gráficos comparando a média das métricas entre diferentes Testes 
    (por exemplo, min1, tc1) para C e Haskell, ignorando o nível de otimização
    na agregação.
    """
    if df.empty:
        print("⚠️ O DataFrame está vazio.")
        return

    metrics = ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]
    metric_labels = {
        "ExecTime_ms": "Tempo de Execução Médio (ms)",
        "MaxWorking_MB": "Memória Física Máx Média (MB)",
        "MaxPrivate_MB": "Memória Privada Máx Média (MB)",
        "AvgCPU_pct": "Uso de CPU Médio (%)",
        "AsmLines": "Linhas Assembly Médias"
    }

    # 1. Agrupar os dados pela Linguagem e pelo Teste para obter a média
    #    (Isso ignora a coluna 'Optimization', calculando a média geral para cada Teste/Linguagem)
    agg_tests = df.groupby(["Test", "Language"])[metrics].mean().reset_index()

    print("=== MÉDIAS AGREGADAS POR TESTE E LINGUAGEM ===")
    print(agg_tests)
    print()

    # 2. Gerar gráficos para cada métrica
    for metric in metrics:
        if metric not in agg_tests.columns:
            continue
            
        # --- Gráfico de Barras para Comparação entre Testes ---
        # Mostrar o crescimento/diferença de uma métrica em diferentes testes
        fig_bar = px.bar(
            agg_tests,
            x="Test",                 # Teste no eixo X
            y=metric,                 # Métrica no eixo Y (a média)
            color="Language",
            barmode="group",
            labels={metric: metric_labels[metric], "Test": "Tipo de Teste"},
            title=f"Comparação Média: {metric_labels[metric]} por Tipo de Teste (C vs. Haskell)",
            # Para 'AsmLines', o crescimento é o que interessa:
            log_y=(metric == "AsmLines") # Pode ser útil para 'AsmLines' se a variação for grande
        )
        fig_bar.update_layout(
            template="plotly_white",
            font=dict(size=14),
            yaxis_title=metric_labels[metric],
            xaxis_title="Tipo de Teste"
        )
        fig_bar.show()

        # --- Gráfico de Linhas (Útil se os 'Testes' tivessem uma ordem lógica) ---
        # Manter o gráfico de linhas caso os testes (e.g., min1, min2, min3) tenham ordem
        fig_line = px.line(
            agg_tests,
            x="Test",
            y=metric,
            color="Language",
            markers=True,
            labels={metric: metric_labels[metric], "Test": "Tipo de Teste"},
            title=f"Comparação Média: {metric_labels[metric]} por Tipo de Teste (C vs. Haskell) - Linha",
        )
        fig_line.update_traces(mode="lines+markers")
        fig_line.update_layout(
            template="plotly_white",
            font=dict(size=14),
            yaxis_title=metric_labels[metric],
            xaxis_title="Tipo de Teste"
        )
        fig_line.show()


# === EXECUÇÃO ===
# Chame a nova função, passando o seu DataFrame completo (df)
compare_tests_aggregated(df)

=== MÉDIAS AGREGADAS POR TESTE E LINGUAGEM ===
           Test Language  ExecTime_ms  MaxWorking_MB  MaxPrivate_MB  \
0       nr_iter        c   141.849099       2.063955       0.441261   
1       nr_iter       py  1448.436818       6.423464       1.369009   
2  nr_recursive        c   150.186514       2.011752       0.432578   
3  nr_recursive       hs   157.874019       2.299131       3.306579   
4  nr_recursive       py   753.265727       6.355264       1.338155   

   AvgCPU_pct  AsmLines  
0    0.305135     135.0  
1    3.644364       0.0  
2    0.254954     172.0  
3    0.946355     576.0  
4    2.754182       0.0  



In [7]:
# === CRIA COLUNA DE TIPO DE TESTE ===
def parse_test_type(test_name):
    if "tc" in test_name.lower():
        return "Time Complexity"
    elif "min" in test_name.lower():
        return "Minimal Complexity"
    elif "ops_complex" in test_name.lower():
        return "Operation Complexity"
    else:
        return "Other"

# ATENÇÃO: se necessário, adicione no fluxo principal:
df["TestType"] = df["Test"].apply(parse_test_type)

def compare_type_clean(test_type):
    # 1. Definição das métricas e rótulos
    metrics = ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]
    metric_labels = {
        "ExecTime_ms": "Tempo de Execução (ms)",
        "MaxWorking_MB": "Memória Física Máx (MB)",
        "MaxPrivate_MB": "Memória Privada Máx (MB)",
        "AvgCPU_pct": "Uso de CPU (%)",
        "AsmLines": "Linhas Assembly"
    }
    
    # 2. Filtra por tipo de teste
    subset = df[df["TestType"] == test_type]
    
    if subset.empty:
        print(f"⚠️ Nenhum dado para '{test_type}'")
        return

    # 3. AGREGAÇÃO: média por (Test, Language)
    agg_subset = subset.groupby(["Test", "Language"])[metrics].mean().reset_index()

    # Exibe um resumo indicando que é a MÉDIA (para deixar claro)
    print(f"=== AGREGADO (MÉDIA) — {test_type} ===")
    print(agg_subset.head())

    # 4. Geração dos Gráficos (com indicação explícita de "média")
    for metric in metrics:
        fig = px.bar(
            agg_subset,
            x="Test",
            y=metric,
            color="Language",
            barmode="group",
            text_auto="%d" if metric == "AsmLines" else ".2s", 
            title=f"✅ {test_type} — {metric_labels[metric]}  — média por teste",
            color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
        )
        
        fig.update_layout(
            template="plotly_white",
            font=dict(size=12),
            xaxis_title="Testes",
            # Marca explicitamente que o eixo Y representa a média
            yaxis_title=f"{metric_labels[metric]} (média)",
            xaxis_tickangle=-45,
            yaxis_type='linear' 
        )
        
        fig.show()


# === EXECUTA PARA TODOS OS TIPOS ===
types = df["TestType"].unique()
for t in types:
    compare_type_clean(t)


=== AGREGADO (MÉDIA) — Other ===
           Test Language  ExecTime_ms  MaxWorking_MB  MaxPrivate_MB  \
0       nr_iter        c   141.849099       2.063955       0.441261   
1       nr_iter       py  1448.436818       6.423464       1.369009   
2  nr_recursive        c   150.186514       2.011752       0.432578   
3  nr_recursive       hs   157.874019       2.299131       3.306579   
4  nr_recursive       py   753.265727       6.355264       1.338155   

   AvgCPU_pct  AsmLines  
0    0.305135     135.0  
1    3.644364       0.0  
2    0.254954     172.0  
3    0.946355     576.0  
4    2.754182       0.0  


In [8]:
# Variáveis e DataFrames (assumindo que 'df', 'metrics', e 'metric_labels' já estão definidos)
metrics = ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]
metric_labels = {
    "ExecTime_ms": "Tempo de Execução (ms)",
    "MaxWorking_MB": "Memória Física Máx (MB)",
    "MaxPrivate_MB": "Memória Privada Máx (MB)",
    "AvgCPU_pct": "Uso de CPU (%)",
    "AsmLines": "Linhas Assembly"
}

# --- 1. Gráfico de Linha: Desempenho (ExecTime) vs. Nível de Otimização, por Linguagem ---

# Objetivo: Mostrar como cada linguagem responde às otimizações.
# Agrega o tempo médio de execução por Nível de Otimização e Linguagem, 
# abrangendo todos os testes.
agg_time_opt = df.groupby(['Optimization', 'Language'])[metrics].mean().reset_index()

fig1 = px.line(
    agg_time_opt,
    x="Optimization",
    y="ExecTime_ms",
    color="Language",
    line_group="Language",
    markers=True,
    title="1. Impacto da Otimização no Tempo de Execução (Média Geral)",
)
fig1.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['ExecTime_ms'],
    xaxis_title="Nível de Otimização",
    font=dict(size=12)
)
fig1.show()


# --- 2. Gráfico de Linha: Complexidade do Código (AsmLines) vs. Nível de Otimização ---

# Objetivo: Mostrar se a otimização reduz ou aumenta o número de instruções geradas.
# Agrega AsmLines por Nível de Otimização e Linguagem.
agg_asm_opt = df.groupby(['Optimization', 'Language'])['AsmLines'].mean().reset_index()

fig2 = px.line(
    agg_asm_opt,
    x="Optimization",
    y="AsmLines",
    color="Language",
    line_group="Language",
    markers=True,
    title="2. Tendência do Número de Linhas Assembly vs. Nível de Otimização",
)
fig2.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['AsmLines'],
    xaxis_title="Nível de Otimização",
    font=dict(size=12)
)
fig2.show()


# --- 3. Gráfico de Linha: Uso de Memória (MaxWorking_MB) ao longo dos Testes ---

# Objetivo: Mostrar o consumo de recursos (Memória) para cada teste, em média.
# Agrega o consumo médio de memória por Teste e Linguagem.
agg_memory_test = df.groupby(['Test', 'Language'])['MaxWorking_MB'].mean().reset_index()

fig3 = px.line(
    agg_memory_test,
    x="Test",
    y="MaxWorking_MB",
    color="Language",
    line_group="Language",
    markers=True,
    title="3. Consumo de Memória (Média) ao Longo da Sequência de Testes",
)
fig3.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['MaxWorking_MB'],
    xaxis_title="Teste (Min1 a Min5 / Tc1 a TcX)",
    xaxis_tickangle=-45,
    font=dict(size=12)
)
fig3.show()

In [9]:
# --- VISUALIZACAO 4: VIOLIN PLOT (Distribuição do Tempo de Execução por Teste) ---

# Objetivo: Mostrar a variabilidade e a densidade do tempo de execução por teste.

fig4 = px.violin(
    df,
    y="AvgCPU_pct",
    x="Test",
    color="Language",
    box=True,  # Inclui box plot dentro do violino
    points="all", # Mostra todos os pontos
    title="4. Distribuição do Uso de CPU (%) por Teste e Linguagem (Violin Plot)",
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
)
fig4.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['AvgCPU_pct'],
    xaxis_title="Teste",
    font=dict(size=12),
    height=1000
)
fig4.show()


In [10]:
# --- VISUALIZACAO 4: VIOLIN PLOT (Distribuição do Tempo de Execução por Teste) ---

# Objetivo: Mostrar a variabilidade e a densidade do tempo de execução por teste.

fig4 = px.violin(
    df,
    y="ExecTime_ms",
    x="Test",
    color="Language",
    box=True,  # Inclui box plot dentro do violino
    points="all", # Mostra todos os pontos
    title="4. Distribuição do Tempo de Execução (ms) por Teste e Linguagem (Violin Plot)",
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
)
fig4.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['ExecTime_ms'],
    xaxis_title="Teste",
    font=dict(size=12),
    height = 1500
)
fig4.show()


In [11]:
# --- VISUALIZACAO 4: VIOLIN PLOT (Distribuição do Tempo de Execução por Teste) ---

# Objetivo: Mostrar a variabilidade e a densidade do tempo de execução por teste.

fig4 = px.violin(
    df,
    y="MaxPrivate_MB",
    x="Test",
    color="Language",
    box=True,  # Inclui box plot dentro do violino
    points="all", # Mostra todos os pontos
    title="4. Distribuição da Memória Privada Máxima (MB) por Teste e Linguagem (Violin Plot)",
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
)
fig4.update_layout(
    template="plotly_white",
    yaxis_title=metric_labels['MaxPrivate_MB'],
    xaxis_title="Teste",
    font=dict(size=12),
    height=1000
)
fig4.show()


In [12]:
# --- VISUALIZACAO 5 (separada): cria um scatter + trendline para cada Test individualmente ---

# Usa a variável `tests` já definida (lista/array de nomes de testes)
for test_name in tests:
    subset = df[df["Test"] == test_name]
    if subset.empty:
        print(f"⚠️ Sem dados para {test_name}")
        continue

    fig = px.scatter(
        subset,
        x="MaxWorking_MB",
        y="ExecTime_ms",
        color="Language",
        trendline="ols",
        title=f"5 — {test_name}: Tempo de Execução vs Memória (por linguagem)",
        labels={"ExecTime_ms": metric_labels["ExecTime_ms"], "MaxWorking_MB": metric_labels["MaxWorking_MB"]},
        color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
    )
    fig.update_layout(
        template="plotly_white",
        font=dict(size=12),
        height=420,
        width=760
    )
    fig.show()


NameError: name 'tests' is not defined

In [ ]:
# --- Preparação: Agregação por Média para o Mapa de Calor ---
heatmap_agg = df.groupby(['Test', 'Language', 'Optimization'])[metrics].mean().reset_index()

# --- VISUALIZACAO 6: HEATMAP - Linhas Assembly vs Teste & Otimização (Geral) ---
asm_heatmap = heatmap_agg.pivot_table(
    index='Test',
    columns=['Language', 'Optimization'],
    values='AsmLines'
)

# Flatten multiindex columns para legendas mais legíveis
asm_heatmap.columns = [f"{lang} — O{opt}" for lang, opt in asm_heatmap.columns]

fig6 = px.imshow(
    asm_heatmap,
    text_auto=True,
    color_continuous_scale="Plasma",
    aspect="auto",
    title="6. Mapa de Calor: Linhas Assembly (Complexidade) por Teste, Linguagem e Otimização",
    labels={'x': 'Linguagem — Nível de Otimização', 'y': 'Teste', 'color': f"{metric_labels['AsmLines']} (média)"}
)

# Ajustes nas legendas/labels/tick para ficar mais claro
fig6.update_layout(
    template="plotly_white",
    xaxis={'side': 'top'},
    height=550,
    font=dict(size=11)
)
fig6.update_xaxes(tickangle=-45)
# Personaliza hover e o título do colorbar (legenda de cor)
fig6.data[0].hovertemplate = 'Linguagem/Opt: %{x}<br>Teste: %{y}<br>Linhas Assembly (média): %{z}<extra></extra>'
fig6.data[0].colorbar.title.text = f"{metric_labels['AsmLines']} (média)"

fig6.show()


In [ ]:
fig = px.scatter(
    df,
    x="Test",
    y="AvgCPU_pct",
    color="Language",
    size="AsmLines",
    hover_data=["Optimization", "TestType", "ExecTime_ms"],
    title="Scatter: Uso Médio de CPU (%) por Teste",
    category_orders={"Test": list(tests)},
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800", "HS": "#00cc96"}
)

fig.update_traces(marker=dict(opacity=0.8, sizemin=6), selector=dict(mode="markers"))
fig.update_layout(
    template="plotly_white",
    xaxis_tickangle=-45,
    yaxis_title=metric_labels["AvgCPU_pct"],
    height=480
)
fig.show()


In [ ]:

# --- VISUALIZACAO 8: BOX PLOT - Variação do Uso de CPU (%) por Nível de Otimização ---

# Objetivo: Mostrar a distribuição (variação) do uso de CPU e identificar outliers.

fig8 = px.box(
    df,
    x="Test",
    y="AvgCPU_pct",
    color="Language",
    points="outliers", # Mostra apenas outliers (pontos extremos)
    title="8. Distribuição do Uso de CPU (%) por Otimização e Linguagem (Box Plot)",
    labels={"AvgCPU_pct": metric_labels["AvgCPU_pct"], "AvgCPU_pct": "Gasto de CPU (%)"},
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
)

fig8.update_layout(
    template="plotly_white",
    font=dict(size=12)
)
fig8.show()

In [ ]:

# --- VISUALIZACAO 8: BOX PLOT - Variação do Uso de CPU (%) por Nível de Otimização ---

# Objetivo: Mostrar a distribuição (variação) do uso de CPU e identificar outliers.

fig8 = px.box(
    df,
    x="Optimization",
    y="AvgCPU_pct",
    color="Language",
    points="all", # Mostra apenas outliers (pontos extremos)
    title="8. Distribuição do Uso de CPU (%) por Otimização e Linguagem (Box Plot)",
    labels={"AvgCPU_pct": metric_labels["AvgCPU_pct"], "Optimization": "Nível de Otimização"},
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"}
)

fig8.update_layout(
    template="plotly_white",
    font=dict(size=12)
)
fig8.show()

In [ ]:
# --- VISUALIZAÇÃO 11: HISTOGRAMA - Distribuição do Tempo de Execução (ms) ---

# Objetivo: Mostrar a frequência de ocorrência dos tempos de execução.
# Usamos 'histfunc="count"' para contar as ocorrências em cada bin.

fig11 = px.histogram(
    df,
    x="ExecTime_ms",
    color="Language",
    marginal="box", # Adiciona um Box Plot na margem para ver a mediana e outliers
    opacity=0.7,
    title="11. Histograma: Distribuição de Frequência do Tempo de Execução (C vs Haskell)",
    labels={"ExecTime_ms": metric_labels["ExecTime_ms"]},
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"},
    # O 'barmode' "overlay" é crucial para sobrepor os histogramas
    barmode="overlay" 
)

fig11.update_layout(
    template="plotly_white",
    yaxis_title="Contagem de Ocorrências",
    font=dict(size=12)
)
fig11.show()

In [ ]:
# --- VISUALIZAÇÃO 12: HISTOGRAMA - Distribuição de Linhas Assembly ---

# Objetivo: Mostrar a frequência de diferentes tamanhos de código gerado.

fig12 = px.histogram(
    df,
    x="AsmLines",
    color="Language",
    marginal="rug", # Adiciona um gráfico de "tapete" na margem para ver a dispersão exata
    opacity=0.7,
    title="12. Histograma: Distribuição da Quantidade de Linhas Assembly (C vs Haskell)",
    labels={"AsmLines": metric_labels["AsmLines"]},
    color_discrete_map={"C": "#0077cc", "Haskell": "#ff8800"},
    barmode="overlay"
)

fig12.update_layout(
    template="plotly_white",
    yaxis_title="Contagem de Ocorrências",
    font=dict(size=12)
)
fig12.show()

In [ ]:
# Agregação necessária para o Heatmap (garante um valor médio Z por X/Y)
heatmap_agg = df.groupby(['Test', 'Optimization', 'Language'])[metrics].mean().reset_index()


# --- VISUALIZAÇÃO 13: HEATMAP - Tempo de Execução vs Teste & Otimização ---

# Objetivo: Mostrar onde o tempo de execução é mais alto/baixo.
# A cor representa o Tempo de Execução (ExecTime_ms).
# O gráfico é facetado pela Linguagem para comparação direta.

fig13 = px.density_heatmap(
    heatmap_agg,
    x="Test",
    y="Optimization",
    z="ExecTime_ms",
    histfunc="sum", # Sumariza o valor de ExecTime_ms
    facet_col="Language", # Separa C e Haskell
    title="13. Heatmap: Tempo de Execução (Média) por Teste e Otimização",
    labels={'ExecTime_ms': metric_labels['ExecTime_ms'], 'Optimization': 'Nível de Otimização'},
    color_continuous_scale=px.colors.sequential.Inferno
)

fig13.update_layout(
    template="plotly_white",
    xaxis={'side': 'bottom'},
    height=400,
    font=dict(size=12)
)
fig13.show()



In [ ]:
df.head()

,File,ExecTime_ms,MaxWorking_MB,MaxPrivate_MB,AvgCPU_pct,AsmFile,AsmLines,Test,Language,Optimization,TestType
0,nr_iter_py.exe,1364.61,6.410,1.352,3.78,o3_nr_iter_py.s,0,nr_iter,py,3,Other
1,nr_iter_py.exe,1356.28,6.406,1.363,3.91,o3_nr_iter_py.s,0,nr_iter,py,3,Other
2,nr_iter_py.exe,1408.82,6.406,1.348,4.02,o3_nr_iter_py.s,0,nr_iter,py,3,Other
3,nr_iter_py.exe,1365.94,6.641,1.367,3.84,o3_nr_iter_py.s,0,nr_iter,py,3,Other
4,nr_iter_py.exe,1317.90,6.402,1.355,3.84,o3_nr_iter_py.s,0,nr_iter,py,3,Other


In [ ]:
# Compact comparison dashboard (new cell)
def make_comparison_plots(df):
    metrics = ["ExecTime_ms", "MaxWorking_MB", "MaxPrivate_MB", "AvgCPU_pct", "AsmLines"]
    tests_list = list(df["Test"].unique())
    opts_list = list(df["Optimization"].unique())

    # 1) Pairwise scatter matrix (quick overview)
    try:
        fig_sm = px.scatter_matrix(
            df,
            dimensions=metrics,
            color="Language",
            title="Pairwise Scatter Matrix — principais métricas",
        )
        fig_sm.update_layout(height=800)
        fig_sm.show()
    except Exception as e:
        print("scatter_matrix failed:", e)

    # 2) Mean bars per Test × Language for ExecTime_ms and AsmLines
    agg = df.groupby(["Test", "Language"])[["ExecTime_ms", "AsmLines"]].mean().reset_index()
    for metric in ["ExecTime_ms", "AsmLines"]:
        fig = px.bar(
            agg,
            x="Test",
            y=metric,
            color="Language",
            barmode="group",
            labels={metric: metric},
            title=f"Média {metric} por Teste e Linguagem"
        )
        fig.update_layout(height=450, xaxis_tickangle=-35)
        fig.show()

    # 3) Scatter: ExecTime vs AsmLines (sizemap by AvgCPU_pct), facet by Optimization
    #    Useful to see trade-offs code size x performance across opt levels
    try:
        fig_sc = px.scatter(
            df,
            x="AsmLines",
            y="ExecTime_ms",
            color="Language",
            size="AvgCPU_pct",
            hover_data=["Test", "Optimization", "File"],
            facet_col="Optimization" if len(opts_list) > 1 else None,
            title="ExecTime vs AsmLines (size=AvgCPU_pct) — por Nível de Otimização"
        )
        fig_sc.update_layout(height=520)
        fig_sc.show()
    except Exception as e:
        print("scatter (ExecTime vs AsmLines) failed:", e)

    # 4) Boxplots to compare distributions by Language (ExecTime and MaxWorking_MB)
    for metric in ["ExecTime_ms", "MaxWorking_MB"]:
        fig_box = px.box(
            df,
            x="Language",
            y=metric,
            color="Language",
            points="outliers",
            title=f"Distribuição de {metric} por Linguagem"
        )
        fig_box.update_layout(height=420)
        fig_box.show()

    # 5) Heatmap (mean ExecTime) Test x Language — visão de alto nível
    heat = df.groupby(["Test", "Language"])["ExecTime_ms"].mean().unstack().fillna(0)
    fig_hm = px.imshow(
        heat,
        text_auto=True,
        aspect="auto",
        labels={"x": "Linguagem", "y": "Teste", "color": "ExecTime_ms (média)"},
        title="Heatmap: ExecTime_ms (média) por Teste × Linguagem"
    )
    fig_hm.update_layout(height=420)
    fig_hm.show()

# Run the dashboard using the existing dataframe `df`
make_comparison_plots(df)

In [ ]:
# New cell: Diverse Plotly visualizations (removed comparisons of AsmLines and any plot driven by counts/quantities)
def variety_of_plotly_charts():
    try:
        # prepare metrics excluding AsmLines
        metrics_no_asm = [m for m in metrics if m != "AsmLines"]

        # 1) 3D Scatter: ExecTime vs MaxWorking_MB vs MaxPrivate_MB (size by AvgCPU_pct)
        if not df.empty:
            fig_3d = px.scatter_3d(
                df.sample(min(1000, len(df))),
                x="ExecTime_ms",
                y="MaxWorking_MB",
                z="MaxPrivate_MB",
                color="Language",
                size="AvgCPU_pct",
                hover_data=["Test", "Optimization", "File"],
                title="3D Scatter: ExecTime vs MaxWorking_MB vs MaxPrivate_MB (size=AvgCPU_pct)"
            )
            fig_3d.update_layout(template="plotly_white", height=700)
            fig_3d.show()
    except Exception as e:
        print("3D scatter failed:", e)

    try:
        # 2) Density Contour: ExecTime vs MaxWorking_MB per Language (no AsmLines)
        fig_contour = px.density_contour(
            df,
            x="ExecTime_ms",
            y="MaxWorking_MB",
            color="Language",
            marginal_x="histogram",
            marginal_y="histogram",
            title="Density Contour: ExecTime vs MaxWorking_MB by Language"
        )
        fig_contour.update_layout(template="plotly_white", height=600)
        fig_contour.show()
    except Exception as e:
        print("density contour failed:", e)

    try:
        # 3) Sunburst: hierarchical view (TestType -> Language -> Optimization) counts (categorical breakdown is OK)
        counts = df.groupby(["TestType", "Language", "Optimization"]).size().reset_index(name="count")
        fig_sun = px.sunburst(
            counts,
            path=["TestType", "Language", "Optimization"],
            values="count",
            title="Sunburst: TestType → Language → Optimization (counts)"
        )
        fig_sun.update_layout(template="plotly_white", height=600)
        fig_sun.show()
    except Exception as e:
        print("sunburst failed:", e)

    try:
        # 4) Treemap: Test → Language → mean ExecTime (area by mean ExecTime, not sample counts)
        treemap_agg = df.groupby(["Test", "Language"]).agg(mean_exec=("ExecTime_ms", "mean")).reset_index()
        if not treemap_agg.empty:
            fig_tree = px.treemap(
                treemap_agg,
                path=["Test", "Language"],
                values="mean_exec",  # use mean exec time as area (avoid raw counts/quantities)
                color="mean_exec",
                color_continuous_scale="Inferno",
                title="Treemap: Test → Language (area=color by mean ExecTime)"
            )
            fig_tree.update_layout(template="plotly_white", height=650)
            fig_tree.show()
    except Exception as e:
        print("treemap failed:", e)

    # Sankey removed to avoid flows based on raw counts/quantities

    try:
        # 5) Parallel Coordinates: mean metrics per Language (normalized) — exclude AsmLines
        if metrics_no_asm:
            mean_per_lang = df.groupby("Language")[metrics_no_asm].mean().reset_index()
            if not mean_per_lang.empty:
                norm = mean_per_lang.copy()
                for m in metrics_no_asm:
                    mn, mx = mean_per_lang[m].min(), mean_per_lang[m].max()
                    norm[m] = (mean_per_lang[m] - (mn if mn is not None else 0)) / ((mx - mn) if (mx != mn) else 1)
                fig_par = px.parallel_coordinates(
                    norm,
                    color=mean_per_lang.index,
                    labels={c: metric_labels.get(c, c) for c in metrics_no_asm},
                    dimensions=metrics_no_asm,
                    title="Parallel Coordinates: Médias das Métricas por Language (normalizado, sem AsmLines)"
                )
                fig_par.update_layout(height=450)
                fig_par.show()
    except Exception as e:
        print("parallel coordinates failed:", e)

    try:
        # 6) Polar (radar-like) plot for mean profile per Language using graph_objects — exclude AsmLines
        if metrics_no_asm:
            mean_data = df.groupby("Language")[metrics_no_asm].mean().reset_index()
            if not mean_data.empty:
                nor = mean_data.copy()
                for m in metrics_no_asm:
                    mn, mx = mean_data[m].min(), mean_data[m].max()
                    nor[m] = 0.5 if (pd.isna(mn) or pd.isna(mx) or mx == mn) else ((mean_data[m] - mn) / (mx - mn))
                fig_pol = go.Figure()
                for _, row in nor.iterrows():
                    fig_pol.add_trace(go.Scatterpolar(
                        r=row[metrics_no_asm].values.tolist(),
                        theta=[metric_labels.get(m, m) for m in metrics_no_asm],
                        fill='toself',
                        name=row['Language']
                    ))
                fig_pol.update_layout(title="Polar: Perfil normalizado por Language (sem AsmLines)", polar=dict(radialaxis=dict(range=[0,1])))
                fig_pol.show()
    except Exception as e:
        print("polar chart failed:", e)

    try:
        # 7) Heatmap: ExecTime mean per Test × Language using heatmap_agg (avoid asm_heatmap)
        if 'heatmap_agg' in globals() and not heatmap_agg.empty:
            try:
                pivot = heatmap_agg.pivot(index="Test", columns="Language", values="ExecTime_ms")
                if pivot is not None and not pivot.empty:
                    fig_h = px.imshow(
                        pivot,
                        text_auto=True,
                        aspect="auto",
                        title="Heatmap: ExecTime médio por Test × Language (sem AsmLines)"
                    )
                    fig_h.update_layout(template="plotly_white", height=450)
                    fig_h.show()
            except Exception:
                # fallback: use simple table heatmap from agg_time_opt
                agg_pivot = agg_time_opt.pivot(index="Optimization", columns="Language", values="ExecTime_ms") if 'agg_time_opt' in globals() else None
                if agg_pivot is not None and not agg_pivot.empty:
                    fig_h = px.imshow(agg_pivot, text_auto=True, aspect="auto", title="Heatmap: ExecTime médio por Opt × Language")
                    fig_h.update_layout(template="plotly_white", height=450)
                    fig_h.show()
    except Exception as e:
        print("imshow heatmap failed:", e)

    try:
        # 8) Scatter Matrix (pairwise) for quick multivariate view (exclude AsmLines)
        if metrics_no_asm:
            fig_sm = px.scatter_matrix(df.sample(min(400, len(df))), dimensions=metrics_no_asm, color="Language",
                                       title="Scatter Matrix (sample) — principais métricas (sem AsmLines)")
            fig_sm.update_layout(height=700)
            fig_sm.show()
    except Exception as e:
        print("scatter matrix failed:", e)

    try:
        # 9) Area chart: ExecTime mean over Tests for each Language (stacked) — unchanged
        agg_test_lang = df.groupby(["Test", "Language"])["ExecTime_ms"].mean().reset_index()
        if not agg_test_lang.empty:
            pivot = agg_test_lang.pivot(index="Test", columns="Language", values="ExecTime_ms").fillna(0)
            fig_area = go.Figure()
            for col in pivot.columns:
                fig_area.add_trace(go.Scatter(
                    x=pivot.index, y=pivot[col],
                    mode='lines',
                    stackgroup='one',
                    name=str(col)
                ))
            fig_area.update_layout(title="Area (stacked): ExecTime médio por Test × Language", xaxis_title="Test", yaxis_title="ExecTime_ms (média)")
            fig_area.show()
    except Exception as e:
        print("area chart failed:", e)

    try:
        # 10) Box + Violin combined per Language for ExecTime (use facets for compact view)
        fig_combo = px.violin(
            df,
            x="Language",
            y="ExecTime_ms",
            box=True,
            points="outliers",
            title="Violin + Box: ExecTime por Language"
        )
        fig_combo.update_layout(template="plotly_white", height=480)
        fig_combo.show()
    except Exception as e:
        print("violin combo failed:", e)

# Execute
variety_of_plotly_charts()
